In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
#conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 14:36:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
from tensorflow.keras.layers import Layer, Input, Conv2D, Conv2DTranspose, ReLU, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf

class WeatherConfig:
    def __init__(self, mean=[], std=[], num_inputs=52, num_hidden1=64, num_hidden2=128, num_outputs=2, kernel_size=(3, 3)):
        self.mean = mean
        self.std = std
        self.num_inputs = num_inputs
        self.num_hidden1 = num_hidden1
        self.num_hidden2 = num_hidden2
        self.num_outputs = num_outputs
        self.kernel_size = kernel_size

class NormalizationLayer(Layer):
    def __init__(self, mean, std, **kwargs):
        super(NormalizationLayer, self).__init__(**kwargs)
        self.mean = tf.constant(mean, dtype=tf.float32)
        self.std = tf.constant(std, dtype=tf.float32)

    def call(self, inputs):
        return (inputs - self.mean) / self.std


2024-04-23 14:36:40.135222: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 14:36:40.549327: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 14:36:42.049187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 14:36:44.916046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def build_model(config):
    model = tf.keras.Sequential([
        Input(shape=(None, None, config.num_inputs)),  # Adjust input shape as needed
        NormalizationLayer(config.mean, config.std),
        Conv2D(config.num_hidden1, config.kernel_size, padding='same', activation='relu'),
        Conv2DTranspose(config.num_hidden2, config.kernel_size, padding='same', activation='relu'),
        Dense(config.num_outputs, activation='relu')  # Assuming the output shape is properly handled
    ])
    return model


In [4]:
from elephas.ml_model import ElephasEstimator

In [5]:
estimator = ElephasEstimator()

In [7]:
config = WeatherConfig()
config

In [8]:
model = build_model(config)
model

RuntimeError: Exception encountered when calling NormalizationLayer.call().

[1mCould not automatically infer the output shape / dtype of 'normalization_layer' (of type NormalizationLayer). Either the `NormalizationLayer.call()` method is incorrect, or you need to implement the `NormalizationLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

Dimensions must be equal, but are 52 and 0 for '{{node sub}} = Sub[T=DT_FLOAT](Placeholder, sub/y)' with input shapes: [?,?,?,52], [0].[0m

Arguments received by NormalizationLayer.call():
  • args=('<KerasTensor shape=(None, None, None, 52), dtype=float32, sparse=None, name=keras_tensor>',)
  • kwargs=<class 'inspect._empty'>